In [1]:
from FiinQuantX import FiinSession, RealTimeData
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables
load_dotenv()
username = os.getenv('USERNAME1')
password = os.getenv('PASSWORD1')

# Login to FiinQuantX
client = FiinSession(username=username, password=password).login()

In [2]:
# This dataframe will be updated by the callback
df_total = pd.DataFrame()

# Define the callback function to handle incoming real-time data
def onEvent(data: RealTimeData):
    global df_total
    try:
        # Cố gắng truy cập các thuộc tính, bao gồm cả 'Time'
        print(f"Received data for: {data.Ticker}, Close: {data.Close}, Time: {data.Time}")
    except AttributeError:
        # Nếu có lỗi, in ra các thuộc tính có sẵn để debug
        # Sử dụng getattr để tránh lỗi nếu 'Ticker' cũng không tồn tại
        ticker = getattr(data, 'Ticker', 'Unknown Ticker')
        print(f"Data received for {ticker}, but some attributes are missing.")
        # In ra các thuộc tính có thể sử dụng được của object 'data'
        attributes = [attr for attr in dir(data) if not attr.startswith('_') and not callable(getattr(data, attr))]
        print(f"Available attributes: {attributes}")

    # Chuyển đổi dữ liệu nhận được sang DataFrame
    # Đảm bảo rằng phương thức to_dataFrame() không gây ra lỗi tương tự
    try:
        new_data_df = data.to_dataFrame()
        # Nối vào DataFrame tổng
        df_total = pd.concat([df_total, new_data_df], ignore_index=True)
    except Exception as e:
        ticker = getattr(data, 'Ticker', 'Unknown Ticker')
        print(f"Could not convert data to DataFrame for {ticker}: {e}")

In [3]:
import time
hose = list(client.TickerList(ticker="VNINDEX"))
hnx = list(client.TickerList(ticker="HNXINDEX"))
upcom = list(client.TickerList(ticker="UPCOMINDEX"))
all_tickers = list(set(hose + hnx + upcom))
print(f"Lấy được {len(all_tickers)} mã chứng khoán từ 3 sàn.")
# Giảm xuống chỉ còn 1 mã để kiểm tra kết nối cơ bản
tickers_to_stream = all_tickers[:1]
print(f"\nSẽ bắt đầu stream dữ liệu cho {len(tickers_to_stream)} mã: {tickers_to_stream}")

TickerEvents = client.Trading_Data_Stream(tickers=tickers_to_stream, callback=onEvent)
TickerEvents.start()

print("\nBắt đầu lắng nghe luồng dữ liệu real-time...")

try:
    while not TickerEvents._stop:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nĐã nhận tín hiệu dừng. Đang đóng kết nối...")
    TickerEvents.stop()
    print("Luồng đã dừng.")

Lấy được 1605 mã chứng khoán từ 3 sàn.

Sẽ bắt đầu stream dữ liệu cho 1 mã: ['CTN']

Bắt đầu lắng nghe luồng dữ liệu real-time...
Connection established. Waiting for server processing...
2025-09-17 08:59:31 Joined group: Realtime.Ticker.CTN

Đã nhận tín hiệu dừng. Đang đóng kết nối...
Disconnecting...
Luồng đã dừng.


In [ ]:
# Sau khi dừng cell xử lý realtime ở trên, bạn có thể chạy cell này để xem lại dữ liệu
if 'df_total' in locals() and not df_total.empty:
    print("Xem 5 dòng đầu của dữ liệu đã thu thập:")
    display(df_total.head())
    print("\nXem 5 dòng cuối của dữ liệu đã thu thập:")
    display(df_total.tail())
else:
    print("Không có dữ liệu 'df_total' để hiển thị. Hãy chạy cell xử lý realtime trước.")


Không có dữ liệu 'df_total' để hiển thị. Hãy chạy cell xử lý realtime trước.
